# IoT Project

##### To get started

We need the following Python libraries:  
- Scapy: For parsing network packet traces
- IPy: For each IP addressing types (what is a local network address vs what is public network address, etcc)
- manuf: Look up a device manufacturer based on MAC address data

In [51]:
!pip install scapy IPy manuf pycurl StringIO

  Could not find a version that satisfies the requirement StringIO (from versions: )
No matching distribution found for StringIO


In [1]:
from scapy.all import *
from manuf import manuf

In [11]:
#help(scapy.all)

In [84]:
packets =rdpcap("./packet_capture.pcap")
p = manuf.MacParser(update=True)
# Example for testing Manufacturer look up based on MAC ID
p.get_all('84:38:35:5a:70:40')[0]
p.get_all('01:00:5e:00:00:fb')

Vendor(manuf=None, comment=None)

In [117]:
import numpy as np
import pandas as pd
from collections import defaultdict
from scapy.all import *
from IPy import IP as IPy
import pprint

d= defaultdict(dict)

for i in range(len(packets)):
#for i in range(500):
    packet = packets[i]
    ## traffic patterns:
    # ARP: ignore for now
    
    # Ignore IPv6 for now
    if (packet.haslayer(IPv6)):
        continue 
    # DNS
    if(packet.haslayer(DNS)):
        dns_type = 'DNS'
        mac_id = packet["Ether.dst"]
        if (packet.haslayer(UDP)):
            if (packet[UDP].sport == 5353):
                dns_type = 'mDNS'
                mac_id = packet["Ether.src"]
        if(packet[DNS].haslayer(DNSRR)):
            rrname = packet[DNS][DNSRR].rrname.decode("utf-8")
            if mac_id not in d:
                d[mac_id] = {
                            'Manufacturer': p.get_all(mac_id)[0],
                            dns_type: rrname
                            }
            else:
                if dns_type in d[mac_id]:
                    d[mac_id][dns_type] += "," + rrname
                else:
                    d[mac_id][dns_type] = rrname 
    # HTTP:
    elif(packet.haslayer(IP)):
        if (packet.haslayer(TCP)):
            mac_id = packet["Ether.src"]
            IP_src = packet["IP.src"]
            IP_dst = packet["IP.dst"]
            if (packet[TCP].dport == 443):
                # This is https, just record high level data
                if ((IPy(IP_src).iptype() == "PRIVATE") and (IPy(IP_dst).iptype() == "PUBLIC")):
                    if mac_id not in d:
                        d[mac_id] = {
                                    'Manufacturer': p.get_all(mac_id)[0],
                                    'HTTPS': {IP_dst}
                                    }
                    else:
                        if 'HTTPS' in d[mac_id]:
                            d[mac_id]['HTTPS'].add(IP_dst)
                        else:
                            d[mac_id]['HTTPS'] = {IP_dst}
            else:
                # This is http, record more data including the payload
                if(len(packet[TCP].payload) > 10):
                    payload = str(packet[TCP].payload)
                else:
                    payload = ""
                if ((IPy(IP_src).iptype() == "PRIVATE") and (IPy(IP_dst).iptype() == "PUBLIC")):
                    if mac_id not in d:
                        d[mac_id] = {
                                    'Manufacturer': p.get_all(mac_id)[0],
                                    'HTTP': {IP_dst: payload}
                                    }
                    else:
                        if 'HTTP' in d[mac_id]:
                            if IP_dst in d[mac_id]['HTTP']:
                                d[mac_id]['HTTP'][IP_dst] +=  payload
                            else:
                                d[mac_id]['HTTP'][IP_dst] = payload 
                        else:
                            d[mac_id]['HTTP']= {IP_dst : payload}  
    # Else: What did I miss
    else:
        None   
    
    # if the device is in the dataframe
    #df.loc[i] = [mac_id, p.get_all(mac_id)[0]]
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(d)

defaultdict(<class 'dict'>,
            {   '00:18:0a:12:2f:2d': {   'HTTP': {   '199.231.78.217': "b'\\x17\\x03\\x01\\x02 "
                                                                       '0<_7\\xf2\\x87\\xf8\\x06\\xafyx\\xaf\\x87u\\xf9;\\x9b\\xb0c,\\xd0\\x0f\\xd1e\\xa6\\x8f\\x92\\x9edN5\\xae\\x99\\xd5\\x02\\xb3\\xda\\x1d\\xe8\\xbfs[\\xb2^\\x00\\xb0\\x95O\\x89\\x8arR\\t\\xe50\\xe7\\xc2\\xf4\\xbeG\\x08\\xce;mTZ\\x90\\x9c\\x01\\xe5\\x88\\x9an\\xff\\xe8wj '
                                                                       '\\x1f\\xafX\\xf0\\xfbs6\\xa2\\x9f/\\x11\\xc1\\x87\\xac\\x93B\\xc5D\\xa0\\xb7\\x8c`l\\xbb\\x14\\x03\\xa2\\x8d\\xd2\\xdc\\xc1x"\\xc6/f\\x02v\\xf3\\x80\\xb4\\xc9n\\x95\\x0c\\xcb\\x17\\xcbxm\\xab\\xa7#+7\\xc6\\x87s\\x11\\xcf\\xdb\\x10\\x17\\xe8\\x7fk\\xec\\x98#\\x83b\\xec\\x16\\xdc\\x86\\x17\\xf2\\xf2\\x99t\\x07o\\xc5\\x8a '
                                                                       "H\\x13o\\xc5K\\xa6\\x97\\x99m\\xd5\\xf6?D_\\xf6+i-\\xad\\x14<A\\xf

In [147]:
known_devices = pd.read_csv('device_types.csv')
known_devices['OUI'] = known_devices['MAC_address'].apply(lambda x: ''.join(x.upper().split(':')[0:3]))
known_devices.head()

,Description,OS,MAC_address,Manufacturer,Device_Type,OUI
0,daghan-macbook-8,Mac OS X 10.13,34:36:3b:7f:3f:a0,Apple,Macbook,34363B
1,iPhone,Apple iPhone,d8:8f:76:bb:35:97,Apple,Iphone,D88F76
2,Daghans-iPhone,Apple iPhone,cc:08:8d:4c:eb:08,Apple,Iphone,CC088D
3,Apple-TV,iOS,50:32:37:a3:4d:38,Apple,TV,503237
4,Agnethas-MacBook-Air,Mac OS X 10.12,84:38:35:5a:70:40,Apple,Macbook,843835


In [131]:
OUI_df = pd.read_csv('oui.csv')
OUI_df.head()

,Registry,Assignment,Organization Name,Organization Address
0,MA-L,E043DB,"Shenzhen ViewAt Technology Co.,Ltd.","9A,Microprofit,6th Gaoxin South Road, High-Tec..."
1,MA-L,2405F5,Integrated Device Technology (Malaysia) Sdn. Bhd.,"Phase 3, Bayan Lepas FIZ Bayan Lepas Penang MY..."
2,MA-L,3CD92B,Hewlett Packard,11445 Compaq Center Drive Houston US 77070
3,MA-L,9C8E99,Hewlett Packard,11445 Compaq Center Drive Houston US 77070
4,MA-L,B499BA,Hewlett Packard,11445 Compaq Center Drive Houston US 77070


In [118]:
dframe = pd.DataFrame.transpose(pd.DataFrame(d))
dframe

,DNS,HTTP,HTTPS,Manufacturer,mDNS
00:18:0a:12:2f:2d,NaN,{'199.231.78.217': 'b'\x17\x03\x01\x02 0<_7\xf...,NaN,CiscoMer,NaN
28:f0:76:31:d3:58,clients6.google.com.,"{'104.154.127.3': 'b""\x93\x938\x1f\x0f\xe5\xb7...","{173.194.203.95, 216.58.194.165, 52.201.158.18...",Apple,NaN
40:cb:c0:bc:36:7e,NaN,NaN,NaN,Apple,"40CBC0BC367E@Apple TV._raop._tcp.local.,40CBC0..."
44:65:0d:90:60:3a,NaN,"{'104.154.127.60': 'b""M]'3\x13\x8c\x93\xbaoO\x...",{52.46.136.77},AmazonTe,NaN
5c:aa:fd:4c:87:a0,NaN,NaN,{54.208.201.219},Sonos,NaN
5c:aa:fd:4c:92:86,NaN,NaN,{54.163.161.36},Sonos,NaN
78:28:ca:03:80:0c,NaN,{'151.101.40.246': 'b'GET /audio/7fd0fe06a28db...,{107.22.76.192},Sonos,NaN
84:38:35:5a:70:40,"googleads.g.doubleclick.net.,cm.g.doubleclick....","{'185.217.0.110': '', '104.154.127.116': 'b'\x...","{172.217.6.65, 172.217.6.33, 199.59.150.42, 74...",Apple,NaN
88:15:44:aa:47:2e,"yahoo.com.,yahoo.com.,yahoo.com.,0.1.0.0.0.0.0...",NaN,NaN,CiscoMer,NaN
88:71:e5:d2:73:4b,NaN,{'104.154.126.239': 'b'\xb2\xf4\xdf\xc1\xac\xe...,"{52.46.132.50, 34.236.127.105}",AmazonTe,NaN


In [76]:
ls(DNS)

length     : ShortField (Cond)                   = (None)
id         : ShortField                          = (0)
qr         : BitField (1 bit)                    = (0)
opcode     : BitEnumField (4 bits)               = (0)
aa         : BitField (1 bit)                    = (0)
tc         : BitField (1 bit)                    = (0)
rd         : BitField (1 bit)                    = (1)
ra         : BitField (1 bit)                    = (0)
z          : BitField (1 bit)                    = (0)
ad         : BitField (1 bit)                    = (0)
cd         : BitField (1 bit)                    = (0)
rcode      : BitEnumField (4 bits)               = (0)
qdcount    : DNSRRCountField                     = (None)
ancount    : DNSRRCountField                     = (None)
nscount    : DNSRRCountField                     = (None)
arcount    : DNSRRCountField                     = (None)
qd         : DNSQRField                          = (None)
an         : DNSRRField                        

In [59]:
import pycurl
from io import StringIO

c = pycurl.Curl()
c.setopt(c.URL, 'https://ipinfo.io/216.58.195.243?token=4b22562280ac95')

e = io.BytesIO()
c.setopt(pycurl.WRITEFUNCTION, e.write)
c.perform()
c.close()
htmlString = e.getvalue().decode('UTF-8')
print(htmlString)

{
  "ip": "216.58.195.243",
  "hostname": "sfo03s06-in-f19.1e100.net",
  "city": "Mountain View",
  "region": "California",
  "country": "US",
  "loc": "37.4192,-122.0570",
  "postal": "94043",
  "org": "AS15169 Google LLC"
}


In [60]:
type(htmlString)

str